In [2]:
import pandas as pd
import numpy as np


In [4]:
df = pd.read_csv("recipe_semi_cleaned_ingredients.csv")
df

In [5]:
df["comments"] = "N/A"

In [6]:
print((df["dish_type"] == "Unknown").sum() / len(df))

0.700566234540307


In [7]:
#ingredients from string that looks like a list to an actual list

df["ingredients"] = df["ingredients"].str.replace("[", "")
df["ingredients"] = df["ingredients"].str.replace("]", "")
df["ingredients"] = df["ingredients"].str.replace("'", "")
df["ingredients"] = df["ingredients"].str.split(", ")

df = df.explode('ingredients')


In [8]:
df["ingredients"] = df["ingredients"].str.strip()
df

In [67]:
#primary cleaning
#convert vulgar fractions
import unicodedata
unicodedata.numeric(u'⅕')
unicodedata.name(u'⅕')

#convert vulgar fractions
for ix, row in df.iterrows():
    for char in row['ingredients']:
        if unicodedata.name(char).startswith('VULGAR FRACTION'):
            normalized = unicodedata.normalize('NFKC', char)
            df.iloc[ix, 3] = df.iloc[ix, 3].replace(char, normalized)

#sanity check for vulgar fractions removal
df.iloc[1:15, :]

,url,country,title,details,ingredients,steps,rating,comments,nutrition,dish_type
0,https://www.allrecipes.com/recipe/81568/afghan...,Afghanistan,Afghan Beef Raviolis (Mantwo),"{'Prep Time:': '45 mins', 'Cook Time:': '1 hr ...",1 teaspoon chopped fresh mint leaves,"['In a small bowl, stir together yogurt, mint,...",4.3,N/A,"{'Calories': '506', 'Fat': '16g', 'Carbs': '57...",Unknown
0,https://www.allrecipes.com/recipe/81568/afghan...,Afghanistan,Afghan Beef Raviolis (Mantwo),"{'Prep Time:': '45 mins', 'Cook Time:': '1 hr ...",2 cloves garlic,"['In a small bowl, stir together yogurt, mint,...",4.3,N/A,"{'Calories': '506', 'Fat': '16g', 'Carbs': '57...",Unknown
0,https://www.allrecipes.com/recipe/81568/afghan...,Afghanistan,Afghan Beef Raviolis (Mantwo),"{'Prep Time:': '45 mins', 'Cook Time:': '1 hr ...",crushed,"['In a small bowl, stir together yogurt, mint,...",4.3,N/A,"{'Calories': '506', 'Fat': '16g', 'Carbs': '57...",Unknown
0,https://www.allrecipes.com/recipe/81568/afghan...,Afghanistan,Afghan Beef Raviolis (Mantwo),"{'Prep Time:': '45 mins', 'Cook Time:': '1 hr ...",1 pound ground beef,"['In a small bowl, stir together yogurt, mint,...",4.3,N/A,"{'Calories': '506', 'Fat': '16g', 'Carbs': '57...",Unknown
0,https://www.allrecipes.com/recipe/81568/afghan...,Afghanistan,Afghan Beef Raviolis (Mantwo),"{'Prep Time:': '45 mins', 'Cook Time:': '1 hr ...",1 ½ cups chopped onion,"['In a small bowl, stir together yogurt, mint,...",4.3,N/A,"{'Calories': '506', 'Fat': '16g', 'Carbs': '57...",Unknown
0,https://www.allrecipes.com/recipe/81568/afghan...,Afghanistan,Afghan Beef Raviolis (Mantwo),"{'Prep Time:': '45 mins', 'Cook Time:': '1 hr ...",1 cup water,"['In a small bowl, stir together yogurt, mint,...",4.3,N/A,"{'Calories': '506', 'Fat': '16g', 'Carbs': '57...",Unknown
0,https://www.allrecipes.com/recipe/81568/afghan...,Afghanistan,Afghan Beef Raviolis (Mantwo),"{'Prep Time:': '45 mins', 'Cook Time:': '1 hr ...",1 carrot,"['In a small bowl, stir together yogurt, mint,...",4.3,N/A,"{'Calories': '506', 'Fat': '16g', 'Carbs': '57...",Unknown
0,https://www.allrecipes.com/recipe/81568/afghan...,Afghanistan,Afghan Beef Raviolis (Mantwo),"{'Prep Time:': '45 mins', 'Cook Time:': '1 hr ...",grated,"['In a small bowl, stir together yogurt, mint,...",4.3,N/A,"{'Calories': '506', 'Fat': '16g', 'Carbs': '57...",Unknown
0,https://www.allrecipes.com/recipe/81568/afghan...,Afghanistan,Afghan Beef Raviolis (Mantwo),"{'Prep Time:': '45 mins', 'Cook Time:': '1 hr ...",¾ teaspoon salt,"['In a small bowl, stir together yogurt, mint,...",4.3,N/A,"{'Calories': '506', 'Fat': '16g', 'Carbs': '57...",Unknown
0,https://www.allrecipes.com/recipe/81568/afghan...,Afghanistan,Afghan Beef Raviolis (Mantwo),"{'Prep Time:': '45 mins', 'Cook Time:': '1 hr ...",1 teaspoon ground black pepper,"['In a small bowl, stir together yogurt, mint,...",4.3,N/A,"{'Calories': '506', 'Fat': '16g', 'Carbs': '57...",Unknown


In [68]:
import spacy
import re
#load the existing small model from spacy
base_model = spacy.load('en_core_web_sm')

import re
import pandas as pd

# Regex for measurements
measurements = re.compile(r'(bowl|bulb|cube|clove|cup|drop|ounce|oz|pinch|pound|teaspoon|tablespoon)s?')
extracted = []

# Loop through each row to extract ingredients
for ix, row in df.iterrows():
    print('\r', "Extracting ingredient for row", ix, end='')
    tokens = base_model(row['ingredients'])
    extract = ''
    for token in tokens:
        if (token.dep_ in ['nsubj', 'ROOT']) and (token.pos_ in ['NOUN', 'PROPN']) and (not measurements.match(token.text)):
            # Explore children
            for child in token.children:
                if (not measurements.match(child.text)) and (child.dep_ in ['amod', 'compound']):
                    extract += child.text + ' '
            extract += token.text + ' '
    extracted.append(extract.strip())  # Use strip() to remove extra spaces

# Replace the original ingredients with the extracted ones
df['ingredients'] = extracted

# Replace empty strings with NaN
nan_value = float("NaN")
df.replace("", nan_value, inplace=True)

# Save the cleaned DataFrame if needed
# df.to_csv("cleanData0.csv", index=False)


/usr/local/lib/python3.11/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


 Extracting ingredient for row 1278

In [69]:
df.dropna(subset = ["ingredients"], inplace=True)

df

,url,country,title,details,ingredients,steps,rating,comments,nutrition,dish_type
0,https://www.allrecipes.com/recipe/81568/afghan...,Afghanistan,Afghan Beef Raviolis (Mantwo),"{'Prep Time:': '45 mins', 'Cook Time:': '1 hr ...",plain yogurt,"['In a small bowl, stir together yogurt, mint,...",4.3,N/A,"{'Calories': '506', 'Fat': '16g', 'Carbs': '57...",Unknown
0,https://www.allrecipes.com/recipe/81568/afghan...,Afghanistan,Afghan Beef Raviolis (Mantwo),"{'Prep Time:': '45 mins', 'Cook Time:': '1 hr ...",ground beef,"['In a small bowl, stir together yogurt, mint,...",4.3,N/A,"{'Calories': '506', 'Fat': '16g', 'Carbs': '57...",Unknown
0,https://www.allrecipes.com/recipe/81568/afghan...,Afghanistan,Afghan Beef Raviolis (Mantwo),"{'Prep Time:': '45 mins', 'Cook Time:': '1 hr ...",water,"['In a small bowl, stir together yogurt, mint,...",4.3,N/A,"{'Calories': '506', 'Fat': '16g', 'Carbs': '57...",Unknown
0,https://www.allrecipes.com/recipe/81568/afghan...,Afghanistan,Afghan Beef Raviolis (Mantwo),"{'Prep Time:': '45 mins', 'Cook Time:': '1 hr ...",carrot,"['In a small bowl, stir together yogurt, mint,...",4.3,N/A,"{'Calories': '506', 'Fat': '16g', 'Carbs': '57...",Unknown
0,https://www.allrecipes.com/recipe/81568/afghan...,Afghanistan,Afghan Beef Raviolis (Mantwo),"{'Prep Time:': '45 mins', 'Cook Time:': '1 hr ...",¾ salt,"['In a small bowl, stir together yogurt, mint,...",4.3,N/A,"{'Calories': '506', 'Fat': '16g', 'Carbs': '57...",Unknown
...,...,...,...,...,...,...,...,...,...,...
1277,https://www.allrecipes.com/recipe/215725/bamie...,Yemen,Bamieh (Middle Eastern Okra Stew),"{'Prep Time:': '20 mins', 'Cook Time:': '1 hr ...",frozen sliced okra,['Heat vegetable oil in a large pot over mediu...,4.4,N/A,"{'Calories': '262', 'Fat': '8g', 'Carbs': '23g...",Unknown
1278,https://www.allrecipes.com/recipe/219483/zimba...,Zimbabwe,Zimbabwean Chicken and Vegetable Soup,"{'Prep Time:': '30 mins', 'Cook Time:': '1 hr ...",olive oil,['Heat the olive oil over medium heat in a lar...,4.7,N/A,"{'Calories': '266', 'Fat': '15g', 'Carbs': '24...",Unknown
1278,https://www.allrecipes.com/recipe/219483/zimba...,Zimbabwe,Zimbabwean Chicken and Vegetable Soup,"{'Prep Time:': '30 mins', 'Cook Time:': '1 hr ...",onion,['Heat the olive oil over medium heat in a lar...,4.7,N/A,"{'Calories': '266', 'Fat': '15g', 'Carbs': '24...",Unknown
1278,https://www.allrecipes.com/recipe/219483/zimba...,Zimbabwe,Zimbabwean Chicken and Vegetable Soup,"{'Prep Time:': '30 mins', 'Cook Time:': '1 hr ...",vegetable stock,['Heat the olive oil over medium heat in a lar...,4.7,N/A,"{'Calories': '266', 'Fat': '15g', 'Carbs': '24...",Unknown


In [70]:
count_df = pd.DataFrame(df.ingredients.value_counts().rename_axis('ingredients').reset_index(name='count'))

print("There are {} unique ingredients".format(count_df.shape[0]))

count_df.head(11)

There are 1702 unique ingredients


,ingredients,count
0,salt,629
1,water,325
2,white sugar,274
3,vegetable oil,253
4,butter,250
5,olive oil,236
6,black pepper,204
7,milk,156
8,purpose flour,133
9,onion,120


In [71]:
recipe_ingredient = df.groupby('title')['ingredients'].apply(set)

ingd_count = {}
for el in count_df.ingredients:
    for r in recipe_ingredient.index:
        if el in recipe_ingredient[r]:
            if el not in ingd_count:
                ingd_count[el] = 1
            else:
                ingd_count[el] += 1


prop_df = pd.DataFrame(ingd_count.items(), columns = ['ingredients', 'proportion'])

prop_df['proportion'] = prop_df['proportion'].div(len(df))

prop_df.sort_values( by = 'proportion', ascending = False)

,ingredients,proportion
0,salt,0.061082
1,water,0.029984
2,white sugar,0.025729
3,vegetable oil,0.024007
4,butter,0.023501
...,...,...
1039,herring fillets,0.000101
1040,whole russet potatoes,0.000101
1041,chili sauce,0.000101
1042,prepared horseradish,0.000101


In [72]:
#join with count_df
results_df = pd.merge(count_df, prop_df, on = 'ingredients')
results_df

#save to file only top 10
results_df.iloc[:10, :].to_csv("results.csv")
#save all results
results_df.to_csv("ingredients2.csv")

In [31]:
results_df

,ingredients,count,proportion
0,salt,629,0.037028
1,water,325,0.018176
2,white sugar,274,0.015597
3,vegetable oil,253,0.014553
4,butter,250,0.014246
...,...,...,...
1697,Indian chili powder,1,0.000061
1698,dried fenugreek leaves,1,0.000061
1699,cauliflower,1,0.000061
1700,fish fillets,1,0.000061


In [74]:
df.to_csv("recipes_igredient_sep2.csv")

In [75]:
country_mapping = {
    "Afghan": "Afghanistan",
    "Albanian": "Albania",
    "Algerian": "Algeria",
    "Andorran": "Andorra",
    "Angolan": "Angola",
    "Antiguan": "Antigua and Barbuda",
    "Argentinian": "Argentina",
    "Armenian": "Armenia",
    "Australian": "Australia",
    "Austrian": "Austria",
    "Azerbaijani": "Azerbaijan",
    "Bahamian": "Bahamas",
    "Bahraini": "Bahrain",
    "Bangladeshi": "Bangladesh",
    "Barbadian": "Barbados",
    "Belarusian": "Belarus",
    "Belgian": "Belgium",
    "Belizean": "Belize",
    "Beninese": "Benin",
    "Bhutanese": "Bhutan",
    "Bolivian": "Bolivia",
    "Bosnian": "Bosnia and Herzegovina",
    "Botswanan": "Botswana",
    "Brazilian": "Brazil",
    "British": "United Kingdom",
    "Bruneian": "Brunei",
    "Bulgarian": "Bulgaria",
    "Burkinabe": "Burkina Faso",
    "Burmese": "Myanmar",
    "Burundian": "Burundi",
    "Cabo Verdean": "Cabo Verde",
    "Cambodian": "Cambodia",
    "Cameroonian": "Cameroon",
    "Canadian": "Canada",
    "Central African": "Central African Republic",
    "Chadian": "Chad",
    "Chilean": "Chile",
    "Chinese": "China",
    "Colombian": "Colombia",
    "Comoran": "Comoros",
    "Congolese": "Congo (Kinshasa)",
    "Costa Rican": "Costa Rica",
    "Croatian": "Croatia",
    "Cuban": "Cuba",
    "Cypriot": "Cyprus",
    "Czech": "Czech Republic",
    "Danish": "Denmark",
    "Djiboutian": "Djibouti",
    "Dominican": "Dominican Republic",
    "Ecuadorian": "Ecuador",
    "Egyptian": "Egypt",
    "Emirati": "United Arab Emirates",
    "Equatoguinean": "Equatorial Guinea",
    "Eritrean": "Eritrea",
    "Estonian": "Estonia",
    "Eswatini": "Eswatini",
    "Ethiopian": "Ethiopia",
    "Fijian": "Fiji",
    "Finnish": "Finland",
    "French": "France",
    "Gabonese": "Gabon",
    "Gambian": "Gambia",
    "Georgian": "Georgia",
    "German": "Germany",
    "Ghanaian": "Ghana",
    "Greek": "Greece",
    "Grenadian": "Grenada",
    "Guatemalan": "Guatemala",
    "Guinean": "Guinea",
    "Guinea-Bissauan": "Guinea-Bissau",
    "Guyanese": "Guyana",
    "Haitian": "Haiti",
    "Honduran": "Honduras",
    "Hungarian": "Hungary",
    "Icelandic": "Iceland",
    "Indian": "India",
    "Indonesian": "Indonesia",
    "Iranian": "Iran",
    "Iraqi": "Iraq",
    "Irish": "Ireland",
    "Israeli": "Israel",
    "Italian": "Italy",
    "Ivorian": "Côte d'Ivoire",
    "Jamaican": "Jamaica",
    "Japanese": "Japan",
    "Jordanian": "Jordan",
    "Kazakhstani": "Kazakhstan",
    "Kenyan": "Kenya",
    "Kiribati": "Kiribati",
    "Kittitian": "Saint Kitts and Nevis",
    "Kosovar": "Kosovo",
    "Kuwaiti": "Kuwait",
    "Kyrgyz": "Kyrgyzstan",
    "Lao": "Laos",
    "Latvian": "Latvia",
    "Lebanese": "Lebanon",
    "Lesotho": "Lesotho",
    "Liberian": "Liberia",
    "Libyan": "Libya",
    "Liechtenstein": "Liechtenstein",
    "Lithuanian": "Lithuania",
    "Luxembourg": "Luxembourg",
    "Madagascan": "Madagascar",
    "Malawian": "Malawi",
    "Malaysian": "Malaysia",
    "Maldivian": "Maldives",
    "Malian": "Mali",
    "Maltese": "Malta",
    "Marshallese": "Marshall Islands",
    "Mauritanian": "Mauritania",
    "Mauritian": "Mauritius",
    "Mexican": "Mexico",
    "Micronesian": "Micronesia",
    "Moldovan": "Moldova",
    "Monacan": "Monaco",
    "Mongolian": "Mongolia",
    "Montenegrin": "Montenegro",
    "Moroccan": "Morocco",
    "Mozambican": "Mozambique",
    "Namibian": "Namibia",
    "Nauruan": "Nauru",
    "Nepalese": "Nepal",
    "Dutch": "Netherlands",
    "New Zealander": "New Zealand",
    "Nicaraguan": "Nicaragua",
    "Nigerian": "Nigeria",
    "Nigerien": "Niger",
    "North Korean": "North Korea",
    "North Macedonian": "North Macedonia",
    "Norwegian": "Norway",
    "Omani": "Oman",
    "Pakistani": "Pakistan",
    "Palauan": "Palau",
    "Palestinian": "Palestine",
    "Panamanian": "Panama",
    "Papua New Guinean": "Papua New Guinea",
    "Paraguayan": "Paraguay",
    "Peruvian": "Peru",
    "Filipino": "Philippines",
    "Polish": "Poland",
    "Portuguese": "Portugal",
    "Qatari": "Qatar",
    "Romanian": "Romania",
    "Russian": "Russia",
    "Rwandan": "Rwanda",
    "Saint Lucian": "Saint Lucia",
    "Salvadoran": "El Salvador",
    "Samoan": "Samoa",
    "San Marinese": "San Marino",
    "Sao Tomean": "Sao Tome and Principe",
    "Saudi": "Saudi Arabia",
    "Senegalese": "Senegal",
    "Serbian": "Serbia",
    "Seychellois": "Seychelles",
    "Sierra Leonean": "Sierra Leone",
    "Singaporean": "Singapore",
    "Slovak": "Slovakia",
    "Slovenian": "Slovenia",
    "Solomon Islander": "Solomon Islands",
    "Somali": "Somalia",
    "South African": "South Africa",
    "South Korean": "South Korea",
    "South Sudanese": "South Sudan",
    "Spanish": "Spain",
    "Sri Lankan": "Sri Lanka",
    "Sudanese": "Sudan",
    "Surinamese": "Suriname",
    "Swazi": "Eswatini",
    "Swedish": "Sweden",
    "Swiss": "Switzerland",
    "Syrian": "Syria",
    "Tajik": "Tajikistan",
    "Tanzanian": "Tanzania",
    "Thai": "Thailand",
    "Timorese": "Timor-Leste",
    "Togolese": "Togo",
    "Tongan": "Tonga",
    "Trinidadian": "Trinidad and Tobago",
    "Tunisian": "Tunisia",
    "Turkish": "Turkey",
    "Turkmen": "Turkmenistan",
    "Tuvaluan": "Tuvalu",
    "Ugandan": "Uganda",
    "Ukrainian": "Ukraine",
    "Uruguayan": "Uruguay",
    "Uzbek": "Uzbekistan",
    "Vanuatuan": "Vanuatu",
    "Venezuelan": "Venezuela",
    "Vietnamese": "Vietnam",
    "Yemeni": "Yemen",
    "Zambian": "Zambia",
    "Zimbabwean": "Zimbabwe"
}

In [76]:
rcp1 = pd.read_csv("/content/recipes_igredient_sep.csv")
rcp2 = pd.read_csv("/content/recipes_igredient_sep2.csv")
rcp1.drop(columns = ["Unnamed: 0"], inplace = True)
rcp2.drop(columns = ["Unnamed: 0"], inplace = True)

In [78]:
rcp1

,url,country,title,details,ingredients,steps,rating,comments,nutrition,dish_type
0,https://www.allrecipes.com/recipe/19344/homema...,Italian,Homemade Lasagna,"{'Prep Time:': '1 hr 25 mins', 'Cook Time:': '...",ground pork,['Gather all ingredients.Dotdash Meredith Food...,4.6,NaN,"{'Calories': '638', 'Fat': '27g', 'Carbs': '54...",Main dishes
1,https://www.allrecipes.com/recipe/19344/homema...,Italian,Homemade Lasagna,"{'Prep Time:': '1 hr 25 mins', 'Cook Time:': '...",lean ground beef,['Gather all ingredients.Dotdash Meredith Food...,4.6,NaN,"{'Calories': '638', 'Fat': '27g', 'Carbs': '54...",Main dishes
2,https://www.allrecipes.com/recipe/19344/homema...,Italian,Homemade Lasagna,"{'Prep Time:': '1 hr 25 mins', 'Cook Time:': '...",minced onion,['Gather all ingredients.Dotdash Meredith Food...,4.6,NaN,"{'Calories': '638', 'Fat': '27g', 'Carbs': '54...",Main dishes
3,https://www.allrecipes.com/recipe/19344/homema...,Italian,Homemade Lasagna,"{'Prep Time:': '1 hr 25 mins', 'Cook Time:': '...",tomato,['Gather all ingredients.Dotdash Meredith Food...,4.6,NaN,"{'Calories': '638', 'Fat': '27g', 'Carbs': '54...",Main dishes
4,https://www.allrecipes.com/recipe/19344/homema...,Italian,Homemade Lasagna,"{'Prep Time:': '1 hr 25 mins', 'Cook Time:': '...",garlic,['Gather all ingredients.Dotdash Meredith Food...,4.6,NaN,"{'Calories': '638', 'Fat': '27g', 'Carbs': '54...",Main dishes
...,...,...,...,...,...,...,...,...,...,...
31342,https://www.allrecipes.com/recipe/244321/heave...,Canadian,Heavenly Raspberry Dessert,"{'Prep Time:': '30 mins', 'Additional Time:': ...",heavy whipping cream,['Butter a 9-inch tube pan. Break angel food c...,4.6,NaN,"{'Calories': '357', 'Fat': '18g', 'Carbs': '47...",Unknown
31343,https://www.allrecipes.com/recipe/245143/mrs-p...,Canadian,Mrs. P's Biscotti,"{'Prep Time:': '15 mins', 'Cook Time:': '30 mi...",white sugar,['Preheat oven to 350 degrees F (175 degrees C...,4.8,NaN,"{'Calories': '65', 'Fat': '2g', 'Carbs': '11g'...",Unknown
31344,https://www.allrecipes.com/recipe/245143/mrs-p...,Canadian,Mrs. P's Biscotti,"{'Prep Time:': '15 mins', 'Cook Time:': '30 mi...",eggs,['Preheat oven to 350 degrees F (175 degrees C...,4.8,NaN,"{'Calories': '65', 'Fat': '2g', 'Carbs': '11g'...",Unknown
31345,https://www.allrecipes.com/recipe/245143/mrs-p...,Canadian,Mrs. P's Biscotti,"{'Prep Time:': '15 mins', 'Cook Time:': '30 mi...",egg yolk,['Preheat oven to 350 degrees F (175 degrees C...,4.8,NaN,"{'Calories': '65', 'Fat': '2g', 'Carbs': '11g'...",Unknown


In [79]:
rcp1["country"] = rcp1["country"].map(country_mapping)
rcp1

,url,country,title,details,ingredients,steps,rating,comments,nutrition,dish_type
0,https://www.allrecipes.com/recipe/19344/homema...,Italy,Homemade Lasagna,"{'Prep Time:': '1 hr 25 mins', 'Cook Time:': '...",ground pork,['Gather all ingredients.Dotdash Meredith Food...,4.6,NaN,"{'Calories': '638', 'Fat': '27g', 'Carbs': '54...",Main dishes
1,https://www.allrecipes.com/recipe/19344/homema...,Italy,Homemade Lasagna,"{'Prep Time:': '1 hr 25 mins', 'Cook Time:': '...",lean ground beef,['Gather all ingredients.Dotdash Meredith Food...,4.6,NaN,"{'Calories': '638', 'Fat': '27g', 'Carbs': '54...",Main dishes
2,https://www.allrecipes.com/recipe/19344/homema...,Italy,Homemade Lasagna,"{'Prep Time:': '1 hr 25 mins', 'Cook Time:': '...",minced onion,['Gather all ingredients.Dotdash Meredith Food...,4.6,NaN,"{'Calories': '638', 'Fat': '27g', 'Carbs': '54...",Main dishes
3,https://www.allrecipes.com/recipe/19344/homema...,Italy,Homemade Lasagna,"{'Prep Time:': '1 hr 25 mins', 'Cook Time:': '...",tomato,['Gather all ingredients.Dotdash Meredith Food...,4.6,NaN,"{'Calories': '638', 'Fat': '27g', 'Carbs': '54...",Main dishes
4,https://www.allrecipes.com/recipe/19344/homema...,Italy,Homemade Lasagna,"{'Prep Time:': '1 hr 25 mins', 'Cook Time:': '...",garlic,['Gather all ingredients.Dotdash Meredith Food...,4.6,NaN,"{'Calories': '638', 'Fat': '27g', 'Carbs': '54...",Main dishes
...,...,...,...,...,...,...,...,...,...,...
31342,https://www.allrecipes.com/recipe/244321/heave...,Canada,Heavenly Raspberry Dessert,"{'Prep Time:': '30 mins', 'Additional Time:': ...",heavy whipping cream,['Butter a 9-inch tube pan. Break angel food c...,4.6,NaN,"{'Calories': '357', 'Fat': '18g', 'Carbs': '47...",Unknown
31343,https://www.allrecipes.com/recipe/245143/mrs-p...,Canada,Mrs. P's Biscotti,"{'Prep Time:': '15 mins', 'Cook Time:': '30 mi...",white sugar,['Preheat oven to 350 degrees F (175 degrees C...,4.8,NaN,"{'Calories': '65', 'Fat': '2g', 'Carbs': '11g'...",Unknown
31344,https://www.allrecipes.com/recipe/245143/mrs-p...,Canada,Mrs. P's Biscotti,"{'Prep Time:': '15 mins', 'Cook Time:': '30 mi...",eggs,['Preheat oven to 350 degrees F (175 degrees C...,4.8,NaN,"{'Calories': '65', 'Fat': '2g', 'Carbs': '11g'...",Unknown
31345,https://www.allrecipes.com/recipe/245143/mrs-p...,Canada,Mrs. P's Biscotti,"{'Prep Time:': '15 mins', 'Cook Time:': '30 mi...",egg yolk,['Preheat oven to 350 degrees F (175 degrees C...,4.8,NaN,"{'Calories': '65', 'Fat': '2g', 'Carbs': '11g'...",Unknown


In [80]:
#stacking add the rcp2 rows to rcp1
rcp1 = pd.concat([rcp1, rcp2], axis=0)
rcp1

,url,country,title,details,ingredients,steps,rating,comments,nutrition,dish_type
0,https://www.allrecipes.com/recipe/19344/homema...,Italy,Homemade Lasagna,"{'Prep Time:': '1 hr 25 mins', 'Cook Time:': '...",ground pork,['Gather all ingredients.Dotdash Meredith Food...,4.6,NaN,"{'Calories': '638', 'Fat': '27g', 'Carbs': '54...",Main dishes
1,https://www.allrecipes.com/recipe/19344/homema...,Italy,Homemade Lasagna,"{'Prep Time:': '1 hr 25 mins', 'Cook Time:': '...",lean ground beef,['Gather all ingredients.Dotdash Meredith Food...,4.6,NaN,"{'Calories': '638', 'Fat': '27g', 'Carbs': '54...",Main dishes
2,https://www.allrecipes.com/recipe/19344/homema...,Italy,Homemade Lasagna,"{'Prep Time:': '1 hr 25 mins', 'Cook Time:': '...",minced onion,['Gather all ingredients.Dotdash Meredith Food...,4.6,NaN,"{'Calories': '638', 'Fat': '27g', 'Carbs': '54...",Main dishes
3,https://www.allrecipes.com/recipe/19344/homema...,Italy,Homemade Lasagna,"{'Prep Time:': '1 hr 25 mins', 'Cook Time:': '...",tomato,['Gather all ingredients.Dotdash Meredith Food...,4.6,NaN,"{'Calories': '638', 'Fat': '27g', 'Carbs': '54...",Main dishes
4,https://www.allrecipes.com/recipe/19344/homema...,Italy,Homemade Lasagna,"{'Prep Time:': '1 hr 25 mins', 'Cook Time:': '...",garlic,['Gather all ingredients.Dotdash Meredith Food...,4.6,NaN,"{'Calories': '638', 'Fat': '27g', 'Carbs': '54...",Main dishes
...,...,...,...,...,...,...,...,...,...,...
9867,https://www.allrecipes.com/recipe/215725/bamie...,Yemen,Bamieh (Middle Eastern Okra Stew),"{'Prep Time:': '20 mins', 'Cook Time:': '1 hr ...",frozen sliced okra,['Heat vegetable oil in a large pot over mediu...,4.4,NaN,"{'Calories': '262', 'Fat': '8g', 'Carbs': '23g...",Unknown
9868,https://www.allrecipes.com/recipe/219483/zimba...,Zimbabwe,Zimbabwean Chicken and Vegetable Soup,"{'Prep Time:': '30 mins', 'Cook Time:': '1 hr ...",olive oil,['Heat the olive oil over medium heat in a lar...,4.7,NaN,"{'Calories': '266', 'Fat': '15g', 'Carbs': '24...",Unknown
9869,https://www.allrecipes.com/recipe/219483/zimba...,Zimbabwe,Zimbabwean Chicken and Vegetable Soup,"{'Prep Time:': '30 mins', 'Cook Time:': '1 hr ...",onion,['Heat the olive oil over medium heat in a lar...,4.7,NaN,"{'Calories': '266', 'Fat': '15g', 'Carbs': '24...",Unknown
9870,https://www.allrecipes.com/recipe/219483/zimba...,Zimbabwe,Zimbabwean Chicken and Vegetable Soup,"{'Prep Time:': '30 mins', 'Cook Time:': '1 hr ...",vegetable stock,['Heat the olive oil over medium heat in a lar...,4.7,NaN,"{'Calories': '266', 'Fat': '15g', 'Carbs': '24...",Unknown


In [81]:
rcp1.to_csv("recipes_igredient_sep_all.csv")

In [82]:
rcp1.columns

Index(['url', 'country', 'title', 'details', 'ingredients', 'steps', 'rating',
       'comments', 'nutrition', 'dish_type'],
      dtype='object')

In [83]:

rcp = pd.read_csv("/content/recipes_igredient_sep_all.csv", index_col = 0)
rcp





,url,country,title,details,ingredients,steps,rating,comments,nutrition,dish_type
0,https://www.allrecipes.com/recipe/19344/homema...,Italy,Homemade Lasagna,"{'Prep Time:': '1 hr 25 mins', 'Cook Time:': '...",ground pork,['Gather all ingredients.Dotdash Meredith Food...,4.6,NaN,"{'Calories': '638', 'Fat': '27g', 'Carbs': '54...",Main dishes
1,https://www.allrecipes.com/recipe/19344/homema...,Italy,Homemade Lasagna,"{'Prep Time:': '1 hr 25 mins', 'Cook Time:': '...",lean ground beef,['Gather all ingredients.Dotdash Meredith Food...,4.6,NaN,"{'Calories': '638', 'Fat': '27g', 'Carbs': '54...",Main dishes
2,https://www.allrecipes.com/recipe/19344/homema...,Italy,Homemade Lasagna,"{'Prep Time:': '1 hr 25 mins', 'Cook Time:': '...",minced onion,['Gather all ingredients.Dotdash Meredith Food...,4.6,NaN,"{'Calories': '638', 'Fat': '27g', 'Carbs': '54...",Main dishes
3,https://www.allrecipes.com/recipe/19344/homema...,Italy,Homemade Lasagna,"{'Prep Time:': '1 hr 25 mins', 'Cook Time:': '...",tomato,['Gather all ingredients.Dotdash Meredith Food...,4.6,NaN,"{'Calories': '638', 'Fat': '27g', 'Carbs': '54...",Main dishes
4,https://www.allrecipes.com/recipe/19344/homema...,Italy,Homemade Lasagna,"{'Prep Time:': '1 hr 25 mins', 'Cook Time:': '...",garlic,['Gather all ingredients.Dotdash Meredith Food...,4.6,NaN,"{'Calories': '638', 'Fat': '27g', 'Carbs': '54...",Main dishes
...,...,...,...,...,...,...,...,...,...,...
9867,https://www.allrecipes.com/recipe/215725/bamie...,Yemen,Bamieh (Middle Eastern Okra Stew),"{'Prep Time:': '20 mins', 'Cook Time:': '1 hr ...",frozen sliced okra,['Heat vegetable oil in a large pot over mediu...,4.4,NaN,"{'Calories': '262', 'Fat': '8g', 'Carbs': '23g...",Unknown
9868,https://www.allrecipes.com/recipe/219483/zimba...,Zimbabwe,Zimbabwean Chicken and Vegetable Soup,"{'Prep Time:': '30 mins', 'Cook Time:': '1 hr ...",olive oil,['Heat the olive oil over medium heat in a lar...,4.7,NaN,"{'Calories': '266', 'Fat': '15g', 'Carbs': '24...",Unknown
9869,https://www.allrecipes.com/recipe/219483/zimba...,Zimbabwe,Zimbabwean Chicken and Vegetable Soup,"{'Prep Time:': '30 mins', 'Cook Time:': '1 hr ...",onion,['Heat the olive oil over medium heat in a lar...,4.7,NaN,"{'Calories': '266', 'Fat': '15g', 'Carbs': '24...",Unknown
9870,https://www.allrecipes.com/recipe/219483/zimba...,Zimbabwe,Zimbabwean Chicken and Vegetable Soup,"{'Prep Time:': '30 mins', 'Cook Time:': '1 hr ...",vegetable stock,['Heat the olive oil over medium heat in a lar...,4.7,NaN,"{'Calories': '266', 'Fat': '15g', 'Carbs': '24...",Unknown


In [84]:
# Create a DataFrame without 'ingredients' where each recipe appears once
recipe_features_df = rcp[[col for col in df.columns if col != 'ingredients']].drop_duplicates()

# Group ingredients for each unique recipe
ingredients_df = rcp[["title", "ingredients"]].groupby('title')['ingredients'].agg(list).reset_index()

final_df = pd.merge(recipe_features_df, ingredients_df, on='title', how='inner')

final_df


,url,country,title,details,steps,rating,comments,nutrition,dish_type,ingredients
0,https://www.allrecipes.com/recipe/19344/homema...,Italy,Homemade Lasagna,"{'Prep Time:': '1 hr 25 mins', 'Cook Time:': '...",['Gather all ingredients.Dotdash Meredith Food...,4.6,NaN,"{'Calories': '638', 'Fat': '27g', 'Carbs': '54...",Main dishes,"[ground pork, lean ground beef, minced onion, ..."
1,https://www.allrecipes.com/recipe/223042/chick...,Italy,Chicken Parmesan,"{'Prep Time:': '15 mins', 'Cook Time:': '20 mi...",['Gather the ingredients. Preheat an oven to 4...,4.8,NaN,"{'Calories': '471', 'Fat': '25g', 'Carbs': '25...",Main dishes,"[skinless, boneless breast halves, salt, large..."
2,https://www.allrecipes.com/recipe/8887/chicken...,Italy,Chicken Marsala,"{'Prep Time:': '10 mins', 'Cook Time:': '20 mi...",['Gather all ingredients.Dotdash Meredith Food...,4.6,NaN,"{'Calories': '448', 'Fat': '27g', 'Carbs': '13...",Main dishes,"[purpose flour, ½ salt, black pepper, medium s..."
3,https://www.allrecipes.com/recipe/222000/spagh...,Italy,Spaghetti Aglio e Olio,"{'Prep Time:': '10 mins', 'Cook Time:': '15 mi...",['Gather all ingredients.Dotdash Meredith Food...,4.7,NaN,"{'Calories': '755', 'Fat': '35g', 'Carbs': '87...",Main dishes,"[uncooked spaghetti, olive oil, pepper flakes,..."
4,https://www.allrecipes.com/recipe/229960/shrim...,Italy,Shrimp Scampi with Pasta,"{'Prep Time:': '20 mins', 'Cook Time:': '20 mi...","['Gather ingredients.', 'Bring a large pot of ...",4.6,NaN,"{'Calories': '511', 'Fat': '19g', 'Carbs': '58...",Main dishes,"[linguine pasta, butter, shallots, shrimp, dry..."
...,...,...,...,...,...,...,...,...,...,...
4585,https://www.allrecipes.com/recipe/27176/vietna...,Vietnam,Vietnamese-Style Chicken Curry Soup,"{'Prep Time:': '30 mins', 'Cook Time:': '2 hrs...",['Heat oil in a large stockpot over medium hea...,4.6,NaN,"{'Calories': '512', 'Fat': '27g', 'Carbs': '41...",Unknown,"[vegetable oil, skin, onion, shallots, stalk l..."
4586,https://www.allrecipes.com/recipe/14087/vietna...,Vietnam,Vietnamese Rice Noodle Salad,"{'Prep Time:': '15 mins', 'Total Time:': '15 m...",['Mince the garlic with the cilantro and the h...,4.4,NaN,"{'Calories': '432', 'Fat': '5g', 'Carbs': '90g...",Unknown,"[packed chopped cilantro, ½ jalapeno pepper, v..."
4587,https://www.allrecipes.com/recipe/218994/nuoc-...,Vietnam,Nuoc Cham (Vietnamese Dipping Sauce),"{'Prep Time:': '15 mins', 'Total Time:': '15 m...",['Stir the sugar and warm water together in a ...,4.7,NaN,"{'Calories': '15', 'Carbs': '4g', 'Protein': '...",Unknown,"[white sugar, warm water, fish sauce, ½ lemon,..."
4588,https://www.allrecipes.com/recipe/215725/bamie...,Yemen,Bamieh (Middle Eastern Okra Stew),"{'Prep Time:': '20 mins', 'Cook Time:': '1 hr ...",['Heat vegetable oil in a large pot over mediu...,4.4,NaN,"{'Calories': '262', 'Fat': '8g', 'Carbs': '23g...",Unknown,"[vegetable oil, large onions, salt, ground cin..."


In [85]:
final_df.to_csv("ingr_cleaned_df.csv")

In [100]:
ingr_by_country = rcp[["country", "ingredients"]]

In [101]:
ingr_by_country.drop_duplicates(inplace = True)
ingr_by_country

<ipython-input-101-0f539bf3bec4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ingr_by_country.drop_duplicates(inplace = True)


,country,ingredients
0,Italy,ground pork
1,Italy,lean ground beef
2,Italy,minced onion
3,Italy,tomato
4,Italy,garlic
...,...,...
9867,Yemen,frozen sliced okra
9868,Zimbabwe,olive oil
9869,Zimbabwe,onion
9870,Zimbabwe,vegetable stock


In [98]:
ingr_by_country.to_csv("ingr_by_country_sep.csv")

In [102]:
import re

# Function to clean ingredient text
def clean_ingredient(ingredient):
    # Remove fractions like ½, ¼, ¾
    ingredient = re.sub(r'[¼½¾⅓⅔⅛⅜⅝⅞]', '', ingredient)

    # Optional: Remove numbers (both standalone and mixed with units)
    ingredient = re.sub(r'\d+[\w/]*', '', ingredient)

    # Remove extra spaces
    ingredient = re.sub(r'\s+', ' ', ingredient).strip()

    return ingredient

# Apply the function to the 'ingredients' column
ingr_by_country['ingredients'] = ingr_by_country['ingredients'].apply(clean_ingredient)

# Check the cleaned DataFrame
print(df.head())


                                                 url      country  \
0  https://www.allrecipes.com/recipe/81568/afghan...  Afghanistan   
0  https://www.allrecipes.com/recipe/81568/afghan...  Afghanistan   
0  https://www.allrecipes.com/recipe/81568/afghan...  Afghanistan   
0  https://www.allrecipes.com/recipe/81568/afghan...  Afghanistan   
0  https://www.allrecipes.com/recipe/81568/afghan...  Afghanistan   

                           title  \
0  Afghan Beef Raviolis (Mantwo)   
0  Afghan Beef Raviolis (Mantwo)   
0  Afghan Beef Raviolis (Mantwo)   
0  Afghan Beef Raviolis (Mantwo)   
0  Afghan Beef Raviolis (Mantwo)   

                                             details   ingredients  \
0  {'Prep Time:': '45 mins', 'Cook Time:': '1 hr ...  plain yogurt   
0  {'Prep Time:': '45 mins', 'Cook Time:': '1 hr ...   ground beef   
0  {'Prep Time:': '45 mins', 'Cook Time:': '1 hr ...         water   
0  {'Prep Time:': '45 mins', 'Cook Time:': '1 hr ...        carrot   
0  {'Prep Time:': 

<ipython-input-102-a3ea06a89d0f>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ingr_by_country['ingredients'] = ingr_by_country['ingredients'].apply(clean_ingredient)


In [104]:
ingr_by_country.to_csv("ingr_by_country_sep_cleaned.csv")